# Extra-1: 추천시스템

SEOYEON CHOI  
2022-12-21

> 추천시스템

# imports

In [1]:
import torch
import numpy as np 
import pandas as pd
from fastai.collab import * 

/home/csy/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

# 나는 솔로

## 주절주절 intro

`-` Data

In [2]:
df_view = pd.read_csv('2022-12-21-rcmdsolo.csv',index_col=0)
df_view 

`-` 데이터를 이해할때 필요한 가정들 – 내맘대로 한 설정임.

-   (옥순,영자,정숙)은 (영식,영철,영호)와 성격이 잘 맞고
    (영숙,순자,현숙)은 (광수,상철,영수)와 성격이 잘맞음
-   ((옥순,영자,정숙),(영식,영철,영호))은 MBTI가 I로 시작하고
    ((영숙,순자,현숙),(광수,상철,영수))는 MBTI가 E로 시작한다.

`-` 목표: NaN 을 추론

`-` 수동추론:

-   (옥순,영호)이 만난다면? $\to$ 둘다 I성향이니까 잘 맞지 않을까? $\to$
    4.0 정도?
-   (정숙,영식)조합은? $\to$ 둘다 I성향이니까 잘 맞지 않을까? + 정숙은
    다 잘맞던데..? $\to$ 4.8 정도?
-   (현숙,영식)조합은? $\to$ 현숙은 E성향인데 영식은 I성향이므로 잘
    안맞을 것임 + 현숙은 원래 좀 눈이 높음 $\to$ 0.25 정도?

`-` 좀 더 체계적인 추론

사람들이 가지고 있는 성향들을 두 개의 숫자로 표현하자.

-   옥순의 성향 = (I성향,E성향) = (1.9, 0.0)
-   영식의 성향 = (I성향,E성향) = (2.0, 0.1)
-   현숙의 성향 = (I성향,E성향) = (0.0, 1.5)

**(1)** 옥순과 영식의 궁합 $\approx$ 옥순의I성향$\times$영식의I성향 $+$
옥순의E성향$\times$영식의E성향 // 적합

In [3]:
a1= np.array([1.9,0.0]).reshape(2,1) # a1은 옥순의 성향, col-vec으로 선언하자. 
b1= np.array([2.0,0.1]).reshape(2,1) # b1은 영식의 성향, col-vec으로 선언하자.
(a1*b1).sum()

3.8

**(2)** 현숙과 영식의 궁합 $\approx$ 현숙의I성향$\times$영식의I성향 $+$
현숙의E성향$\times$영식의E성향 // 예측

In [4]:
a6= np.array([0.0,1.5]).reshape(2,1)
(a6*b1).sum()

0.15000000000000002

-   그럴듯함..

`-` 모델링

아래가 같음을 관찰하라. (차원만 다름)

In [5]:
(a1*b1).sum(), a1.T@b1

(3.8, array([[3.8]]))

In [6]:
(a6*b1).sum(), a6.T@b1

(0.15000000000000002, array([[0.15]]))

만약에 여자의성향, 남자의성향을 적당한 매트릭스로 정리할 수 있다면
궁합매트릭스를 만들 수 있음

In [7]:
a1= np.array([1.9,0.0]).reshape(2,1)
a2= np.array([2.0,0.1]).reshape(2,1)
a3= np.array([2.5,1.0]).reshape(2,1)
a4= np.array([0.1,1.9]).reshape(2,1)
a5= np.array([0.2,2.1]).reshape(2,1)
a6= np.array([0.0,1.5]).reshape(2,1)
A = np.concatenate([a1,a2,a3,a4,a5,a6],axis=1)
A

array([[1.9, 2. , 2.5, 0.1, 0.2, 0. ],
       [0. , 0.1, 1. , 1.9, 2.1, 1.5]])

In [8]:
b1= np.array([2.0,0.1]).reshape(2,1)
b2= np.array([1.9,0.2]).reshape(2,1)
b3= np.array([1.8,0.3]).reshape(2,1)
b4= np.array([0.3,2.1]).reshape(2,1)
b5= np.array([0.2,2.0]).reshape(2,1)
b6= np.array([0.1,1.9]).reshape(2,1)
B = np.concatenate([b1,b2,b3,b4,b5,b6],axis=1)
B

array([[2. , 1.9, 1.8, 0.3, 0.2, 0.1],
       [0.1, 0.2, 0.3, 2.1, 2. , 1.9]])

In [9]:
A.T@B

array([[3.8 , 3.61, 3.42, 0.57, 0.38, 0.19],
       [4.01, 3.82, 3.63, 0.81, 0.6 , 0.39],
       [5.1 , 4.95, 4.8 , 2.85, 2.5 , 2.15],
       [0.39, 0.57, 0.75, 4.02, 3.82, 3.62],
       [0.61, 0.8 , 0.99, 4.47, 4.24, 4.01],
       [0.15, 0.3 , 0.45, 3.15, 3.  , 2.85]])

In [10]:
a1.T@b1, a2.T@b2, a3.T@b1

(array([[3.8]]), array([[3.82]]), array([[5.1]]))

결국 모형은 아래와 같다.

$$\text{궁합매트릭스} = {\bf A}^\top {\bf B} + \text{오차}$$

`-` 학습전략: 아래의 매트릭스중에서 어떤값은 관측하였고 어떤값은
관측하지 못함 $\to$ 관측한 값들만 대충 비슷하게 하면 되는거 아니야?

In [11]:
A.T@B 

array([[3.8 , 3.61, 3.42, 0.57, 0.38, 0.19],
       [4.01, 3.82, 3.63, 0.81, 0.6 , 0.39],
       [5.1 , 4.95, 4.8 , 2.85, 2.5 , 2.15],
       [0.39, 0.57, 0.75, 4.02, 3.82, 3.62],
       [0.61, 0.8 , 0.99, 4.47, 4.24, 4.01],
       [0.15, 0.3 , 0.45, 3.15, 3.  , 2.85]])

In [12]:
df_view

`-` 자료를 아래와 같이 정리한다면?

In [13]:
df = pd.DataFrame([(f,m,df_view.loc[f,m]) for f in df_view.index for m in df_view.columns if not np.isnan(df_view.loc[f,m])])
df.columns = ['X1','X2','y']
df

In [14]:
mapp1 = {k[1]:k[0] for k in enumerate(df.X1.unique())}
mapp2 = {k[1]:k[0] for k in enumerate(df.X2.unique())}
mapp1,mapp2

({'옥순': 0, '영자': 1, '정숙': 2, '영숙': 3, '순자': 4, '현숙': 5},
 {'영식': 0, '영철': 1, '광수': 2, '상철': 3, '영호': 4, '영수': 5})

In [15]:
X1 = torch.tensor(list(map(lambda name: mapp1[name], df.X1)))
X2 = torch.tensor(list(map(lambda name: mapp2[name], df.X2)))
X1 = torch.nn.functional.one_hot(X1).float()
X2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df.y).float()

`-` yhat을 구하는 과정..

In [16]:
l1 = torch.nn.Linear(in_features=6,out_features=2) 
l2 = torch.nn.Linear(in_features=6,out_features=2)

In [17]:
l1(X1) # 옥순~현숙의 성향들 

tensor([[ 0.5101,  0.1558],
        [ 0.5101,  0.1558],
        [ 0.5101,  0.1558],
        [ 0.5101,  0.1558],
        [ 0.1860, -0.1332],
        [ 0.1860, -0.1332],
        [ 0.1860, -0.1332],
        [ 0.1860, -0.1332],
        [ 0.5840, -0.5035],
        [ 0.5840, -0.5035],
        [ 0.5840, -0.5035],
        [ 0.5840, -0.5035],
        [ 0.0356, -0.3052],
        [ 0.0356, -0.3052],
        [ 0.0356, -0.3052],
        [ 0.0356, -0.3052],
        [ 0.5411, -0.3360],
        [ 0.5411, -0.3360],
        [ 0.5411, -0.3360],
        [ 0.5411, -0.3360],
        [ 0.0738,  0.1614],
        [ 0.0738,  0.1614],
        [ 0.0738,  0.1614],
        [ 0.0738,  0.1614]], grad_fn=<AddmmBackward0>)

In [18]:
l2(X2) # 영식~영수의 성향들 

tensor([[ 0.1939,  0.0405],
        [ 0.1073, -0.2484],
        [-0.0011, -0.2328],
        [-0.1622, -0.1059],
        [ 0.1939,  0.0405],
        [-0.1635, -0.0460],
        [-0.0011, -0.2328],
        [-0.1319,  0.1783],
        [ 0.1073, -0.2484],
        [-0.1635, -0.0460],
        [-0.1622, -0.1059],
        [-0.1319,  0.1783],
        [ 0.1939,  0.0405],
        [ 0.1073, -0.2484],
        [-0.0011, -0.2328],
        [-0.1622, -0.1059],
        [ 0.1939,  0.0405],
        [ 0.1073, -0.2484],
        [-0.0011, -0.2328],
        [-0.1319,  0.1783],
        [ 0.1073, -0.2484],
        [-0.1635, -0.0460],
        [-0.1622, -0.1059],
        [-0.1319,  0.1783]], grad_fn=<AddmmBackward0>)

`-` 몇개의 관측치만 생각해보자..

In [19]:
df.head()

In [20]:
(l1(X1)[0]*l2(X2)[0]).sum() # (옥순의성향 * 영식의성향).sum()

tensor(0.1052, grad_fn=<SumBackward0>)

-   이 값이 실제로는 3.9 이어야 한다.

In [21]:
(l1(X1)[1]*l2(X2)[1]).sum() # (옥순의성향 * 영철의성향).sum()

tensor(0.0160, grad_fn=<SumBackward0>)

-   이 값이 실제로는 4.1 이어야 한다.

`-` yhat을 구하면!

In [22]:
yhat = (l1(X1) * l2(X2)).sum(axis=1) # (l1(X1) * l2(X2)).sum(1)와 결과가 같음 
yhat

tensor([ 0.1052,  0.0160, -0.0369, -0.0992,  0.0307, -0.0243,  0.0308, -0.0483,
         0.1877, -0.0724, -0.0414, -0.1668, -0.0054,  0.0796,  0.0710,  0.0265,
         0.0913,  0.1415,  0.0776, -0.1313, -0.0322, -0.0195, -0.0291,  0.0190],
       grad_fn=<SumBackward1>)

In [23]:
yhat[:2],y[:2] # 이 값들이 비슷해야 하는데..

(tensor([0.1052, 0.0160], grad_fn=<SliceBackward0>), tensor([3.9000, 4.1000]))

`-` 0~5 까지의 범위로 고정되어 있으니까 아래와 같이 해도 되겠음..

In [24]:
sig = torch.nn.Sigmoid()

In [25]:
yhat = sig((l1(X1) * l2(X2)).sum(axis=1))*5 # (l1(X1) * l2(X2)).sum(1)와 결과가 같음 
yhat

tensor([2.6314, 2.5200, 2.4539, 2.3760, 2.5383, 2.4696, 2.5385, 2.4397, 2.7340,
        2.4096, 2.4482, 2.2920, 2.4932, 2.5995, 2.5887, 2.5332, 2.6140, 2.6766,
        2.5970, 2.3361, 2.4598, 2.4756, 2.4637, 2.5238],
       grad_fn=<MulBackward0>)

In [26]:
loss = torch.mean((y-yhat)**2)
loss

tensor(3.2296, grad_fn=<MeanBackward0>)

## torch를 이용한 학습

In [27]:
torch.manual_seed(43052)
l1 = torch.nn.Linear(6,2) 
l2 = torch.nn.Linear(6,2)
sig = torch.nn.Sigmoid() 

In [28]:
loss_fn = torch.nn.MSELoss() 
optimizr = torch.optim.Adam(list(l1.parameters())+list(l2.parameters()))

In [29]:
for epoc in range(5000):
    ## 1 
    feature1 = l1(X1)
    feature2 = l2(X2) 
    matching_score = (feature1*feature2).sum(axis=1) 
    yhat = sig(matching_score)*5 # 만약에 1~3점이라면 "1+sig(matching_score)*2" 와 같이 하면 되었을듯 
    ## 2 
    loss = loss_fn(yhat,y)    
    ## 3 
    loss.backward()    
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [30]:
yhat

tensor([3.9382, 4.0624, 0.4665, 0.3353, 4.5038, 3.6975, 0.3562, 0.3558, 4.8614,
        4.7208, 1.1813, 1.3158, 0.4606, 0.3573, 4.1288, 4.2734, 0.8611, 0.7347,
        4.0493, 4.0464, 0.1810, 0.3124, 3.5031, 3.3948],
       grad_fn=<MulBackward0>)

In [31]:
y

tensor([3.9000, 4.1000, 0.5000, 0.3000, 4.5000, 3.7000, 0.5000, 0.2000, 4.9000,
        4.7000, 1.2000, 1.3000, 0.6000, 0.2000, 4.1000, 4.3000, 0.7000, 0.9000,
        4.2000, 3.9000, 0.2000, 0.3000, 3.5000, 3.4000])

In [32]:
l1(X1) # 두번째 칼럼이 I 성향 점수로 "해석"된다

tensor([[-1.4663,  0.2938],
        [-1.4663,  0.2938],
        [-1.4663,  0.2938],
        [-1.4663,  0.2938],
        [-1.7086,  0.6597],
        [-1.7086,  0.6597],
        [-1.7086,  0.6597],
        [-1.7086,  0.6597],
        [-0.8705,  1.2945],
        [-0.8705,  1.2945],
        [-0.8705,  1.2945],
        [-0.8705,  1.2945],
        [ 1.1046, -0.8298],
        [ 1.1046, -0.8298],
        [ 1.1046, -0.8298],
        [ 1.1046, -0.8298],
        [ 0.9880, -0.5193],
        [ 0.9880, -0.5193],
        [ 0.9880, -0.5193],
        [ 0.9880, -0.5193],
        [ 0.6834, -1.2201],
        [ 0.6834, -1.2201],
        [ 0.6834, -1.2201],
        [ 0.6834, -1.2201]], grad_fn=<AddmmBackward0>)

-   포인트: 여성출연자중, 정숙은 대체로 잘 맞춰주고 현숙은 그렇지
    않았음.. $\to$ 그러한 가중치가 잘 드러남!!

## fastai를 이용한 학습

`(1)` dls

In [33]:
df.head() # 앞단계 전처리의 산물

In [34]:
dls = CollabDataLoaders.from_df(df,bs=2,valid_pct=2/24)

`(2)` lrnr 생성

In [35]:
lrnr = collab_learner(dls,n_factors=2,y_range=(0,5))

`(3)` 학습

In [36]:
lrnr.fit(30,lr=0.05)

`(4)` 예측

**적합값 확인**

In [37]:
lrnr.show_results()

**(옥순의 궁합)**

In [38]:
df_new = pd.DataFrame({'X1':['옥순']*6, 'X2':['영식','영철','영호','광수','상철','영수']})
df_new

In [39]:
lrnr.get_preds(dl=dls.test_dl(df_new))

(tensor([3.9011, 4.1288, 3.6021, 0.6609, 0.3678, 0.5451]), None)

비교를 위해서

In [40]:
df_view

**(정숙의 궁합)**

In [41]:
df_new = pd.DataFrame({'X1':['정숙']*6, 'X2':['영식','영철','영호','광수','상철','영수']})
df_new

In [42]:
lrnr.get_preds(dl=dls.test_dl(df_new))

(tensor([4.8320, 4.8423, 4.6991, 1.5996, 1.1552, 1.2886]), None)

높으면 잘 맞는다,

비교를 위해서

In [43]:
df_view

`-` Appedix: fastai 구조공부..

In [44]:
lrnr.model

EmbeddingDotBias(
  (u_weight): Embedding(7, 2)
  (i_weight): Embedding(7, 2)
  (u_bias): Embedding(7, 1)
  (i_bias): Embedding(7, 1)
)

In [45]:
lrnr.model.forward??

Signature: lrnr . model . forward ( x ) 
 Docstring: 
Defines the computation performed at every call.
Should be overridden by all subclasses.
.. note::
 Although the recipe for forward pass needs to be defined within
 this function, one should call the :class:`Module` instance afterwards
 instead of this since the former takes care of running the
 registered hooks while the latter silently ignores them.
 Source: 
 def forward ( self , x ) : 
 users , items = x [ : , 0 ] , x [ : , 1 ] 
 dot = self . u_weight ( users ) * self . i_weight ( items ) 
 res = dot . sum ( 1 ) + self . u_bias ( users ) . squeeze ( ) + self . i_bias ( items ) . squeeze ( ) 
 if self . y_range is None : return res
 return torch . sigmoid ( res ) * ( self . y_range [ 1 ] - self . y_range [ 0 ] ) + self . y_range [ 0 ] 
 File: ~/anaconda3/envs/py37/lib/python3.7/site-packages/fastai/collab.py
 Type: method

-   bias를 제외하면 우리가 짠 모형과 같음!

# 커피 or 홍차

## data

`-` 예전에 살펴본 예제

In [46]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_anal.csv')
df

1000 rows × 4 columns

`-` 기억을 살리기 위해서..

In [47]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2022/main/posts/I.%20Overview/2022-09-08-rcmd_view.csv')
df_view

100 rows × 20 columns

## 모형

(편의상 바이어스를 제외하면)

`-` 특징벡터:

-   유저1의 취향 = \[커피를 좋아하는 정도, 홍차를 좋아하는 정도\]
-   아이템1의 특징 = \[커피의 특징, 홍차인 특징\]

`-` 평점

-   유저1이 아이템1을 먹었을경우 평점: 유저1의 취향과 아이템1의 특징의
    내적 = (유저1의 취향 $\odot$ 아이템1의 특징).sum()

## 학습

`(1)` dls

In [48]:
dls = CollabDataLoaders.from_df(df)

In [49]:
dls.items

800 rows × 4 columns

`(2)` lrnr

In [50]:
lrnr = collab_learner(dls,n_factors=2) # 교재에는 y_range 를 설정하도록 되어있지만 설정 안해도 적합에는 크게 상관없음..

`(3)` fit

In [51]:
lrnr.fit(10,0.1)

`(4)` predict

(적합된 값 확인)

In [52]:
lrnr.show_results() # 누를때마다 결과다름

(예측값)

In [53]:
df_new = pd.DataFrame({'user':[1,1,1,1], 'item':[9,10,11,12]})
df_new

In [54]:
lrnr.get_preds(dl=dls.test_dl(df_new))

(tensor([3.9138, 3.9825, 0.9447, 0.8223]), None)